Transfer Learning with small Data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
%cd '/content/gdrive/My Drive/...'

In [ ]:
#import libs
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dropout,Flatten,Dense,GlobalMaxPooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler,TensorBoard,EarlyStopping


In [ ]:
#Small dataset ~300 images: 2 classes -> strategy 4

In [ ]:
img_width, img_height = 224, 224
train_data_dir = "dataset/training_set"
validation_data_dir = "dataset/test_set"

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16
model = applications.VGG16(weights = "imagenet",
include_top=False, #whether to include the 3 fully-connected
input_shape = (img_width, img_height, 3))

In [ ]:
model.summary()

In [ ]:
#New dataset is small and similar to original dataset

In [ ]:
# So lets freeze all the VGG16 layers and train only the classifier
for layer in model.layers:
  layer.trainable = False

In [ ]:
#Adding custom Layers - ANN
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

In [ ]:
# creating the final model
model_final = Model(inputs = model.input, outputs = predictions)

In [ ]:
# compile the model
model_final.compile(loss = "categorical_crossentropy",
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),
                    metrics=["accuracy"])

In [ ]:
# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True,
                                  fill_mode = "nearest",
                                  zoom_range = 0.3,
                                  width_shift_range = 0.3,
                                  height_shift_range=0.3,
                                  rotation_range=30)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height,
                                                    img_width),
                                                    batch_size = 32,
                                                    class_mode = "categorical")
validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size = (img_height,
                                                        img_width),
                                                        class_mode = "categorical")

In [ ]:
# Save the model according to the conditions
checkpoint = ModelCheckpoint("vgg16_1.h5",
                              monitor='val_loss',
                              verbose=1,
                              save_best_only=True,
                              save_weights_only=False,
                              mode='auto',
                              save_freq=1)
early = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=10,
                      verbose=1,
                      mode='auto')

In [ ]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
print(t0)

In [ ]:
# Train the model
model_final.fit(train_generator,
                steps_per_epoch =len(train_generator),
                epochs = 100,
                validation_data = validation_generator,
                validation_steps = len(validation_generator),
                verbose = 1,
                callbacks = [checkpoint, early])

In [ ]:
t1 = datetime.datetime.now()
print(t1-t0)

In [ ]:
# save model
from tensorflow.keras.models import load_model
# Creates a HDF5 file 'my_model.h5'
model_final.save('my_vgg16.h5')
print("save!!!")

In [ ]:
#Making new predictions
from tensorflow.keras.models import load_model
classifier = load_model('my_vgg16.h5')
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
test_image = image.load_img('dataset/single_prediction/cat2.jpg',
target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
print(result)
#training_set.class_indices
if result[0][0] > result[0][1]:
  prediction = 'cat'
else:
  prediction = 'dog'
print(prediction)

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('dataset/single_prediction/cat2.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
test_image = image.load_img('dataset/single_prediction/dog1.jpg',
                            target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
print(result)
#training_set.class_indices
if result[0][0] > result[0][1]:
  prediction = 'cat'
else:
  prediction = 'dog'
print(prediction)

In [ ]:
img=mpimg.imread('dataset/single_prediction/dog1.jpg')
imgplot = plt.imshow(img)
plt.show()